# Chapter 5: Formatting Output and Speaking for Claude

（第5章：出力フォーマットと “Claude の代わりに話す”）

- [レッスン](#lesson)
- [演習](#exercises)
- [例のプレイグラウンド](#example-playground)

## Setup

次のセットアップセルを実行して、APIキーの読み込みと `get_completion` ヘルパー関数の準備を行います。

In [15]:
# %pip install anthropic --quiet

# Import the hints module from the utils package
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# Import python's built-in regular expression library
import re
from anthropic import AnthropicBedrock

%store -r MODEL_NAME
%store -r AWS_REGION
MODEL_NAME = "us.anthropic.claude-3-haiku-20240307-v1:0"


client = AnthropicBedrock(aws_region=AWS_REGION)

def get_completion(prompt, system='', prefill=''):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ],
        system=system
    )
    return message.content[0].text

---

## Lesson

**Claude は、さまざまな形式で出力をフォーマットできます。** そのようにしてほしいと頼むだけです。

方法のひとつが、XML タグを使って、応答本体を余計なテキストから分離することです。すでに XML タグを使うとプロンプトが明確になり、Claude が解析しやすくなることを学びました。同様に、Claude に **出力も XML タグで包むよう依頼** することで、人間にとっても読みやすく整理された出力にできます。

### Examples

第2章で、Claude に前置き（preamble）を省いて詩に直接入るよう頼むことで『詩の前置き問題』を解決しました。実は、**詩を XML タグに入れるよう指示する** ことでも似た効果が得られます。

In [3]:
# Variable content
# ANIMAL = "Rabbit"
ANIMAL = "ウサギ"

# Prompt template with a placeholder for the variable content
# PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."
PROMPT = f"{ANIMAL}についての俳句を作成してください。<haiku>タグで囲んでください。"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

--------------------------- Full prompt with variable substutions ---------------------------
ウサギについての俳句を作成してください。<haiku>タグで囲んでください。

------------------------------------- Claude's response -------------------------------------
<haiku>
白いウサギ
ぴょんぴょん跳ねて
春の野原
</haiku>


なぜこれが役立つのでしょうか？ 出力を **XML タグで包むと、短いプログラムで XML タグ間の内容だけを抽出することで『詩だけ』を確実に取り出せる** からです。

このテクニックを発展させたものとして、**最初の XML タグを `assistant` ターンに入れる** 方法があります。`assistant` ターンにテキストを入れるのは、『Claude がすでにそこまで言った』ことにして、そこから続けるよう促すのと同じです。この手法は "speaking for Claude"（Claude の代わりに話す）や "prefilling Claude's response"（応答の先頭をあらかじめ埋める）と呼ばれます。

下の例では、最初の `<haiku>` タグを先に入れています。Claude がそこから直接続きを出力していることに注目してください。

In [ ]:
# Variable content
ANIMAL = "Cat"
ANIMAL = "Cat"
ANIMAL = "猫"

# Prompt template with a placeholder for the variable content
# PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."
PROMPT = f"{ANIMAL}についての俳句を作成してください。<haiku>タグで囲んでください。"

# Prefill for Claude's response
PREFILL = "<haiku>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN:
猫についての俳句を作成してください。<haiku>タグで囲んでください。

ASSISTANT TURN:
<haiku>

------------------------------------- Claude's response -------------------------------------

陽だまりで
毛づくろいする
猫の午後
</haiku>


Claude は、ほかの出力形式、特に `JSON` も得意です。JSON 形式を（完全に決定的ではないものの）かなり高い確度で守らせたい場合は、`assistant` 側を `{` のような開始括弧で事前に埋めることもできます。

In [7]:
# Variable content
# ANIMAL = "Cat"
ANIMAL = "猫"

# Prompt template with a placeholder for the variable content
# PROMPT = f"Please write a haiku about {ANIMAL}. Use JSON format with the keys as \"first_line\", \"second_line\", and \"third_line\"."
PROMPT = f"{ANIMAL}についての俳句をJSON形式で作成してください。キーは\"first_line\"、\"second_line\"、\"third_line\"としてください。"
# PROMPT = f"{ANIMAL}についての俳句をJSON形式で作成してください。"

# Prefill for Claude's response
PREFILL = "{"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
猫についての俳句をJSON形式で作成してください。キーは"first_line"、"second_line"、"third_line"としてください。

ASSISTANT TURN
{

------------------------------------- Claude's response -------------------------------------

  "first_line": "窓辺にて",
  "second_line": "猫がまどろむ午後",
  "third_line": "陽だまりかな"
}


以下は、**同じプロンプト内で複数の入力変数を使い、さらに出力フォーマット指定もすべて XML タグで行う** 例です。

In [ ]:
# First input variable
# EMAIL = "Hi Zack, just pinging you for a quick update on that prompt you were supposed to write."
EMAIL = "こんにちは、ザック。あなたが書くはずだったプロンプトの最新情報を簡単にお知らせするために、連絡しました。"

# Second input variable
ADJECTIVE = "olde english"
# ADJECTIVE = "古い英語"

# Prompt template with a placeholder for the variable content
# PROMPT = f"Hey Claude. Here is an email: <email>{EMAIL}</email>. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE}_email> XML tags."
PROMPT = f"こんにちは、Claude。ここにメールがあります：<email>{EMAIL}</email>。このメールを{ADJECTIVE}にしてください。新しいバージョンを<{ADJECTIVE}_email> XMLタグで書いてください。"

# Prefill for Claude's response (now as an f-string with a variable)
PREFILL = f"<{ADJECTIVE}_email>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
こんにちは、Claude。ここにメールがあります：<email>こんにちは、ザック。あなたが書くはずだったプロンプトの最新情報を簡単にお知らせするために、連絡しました。</email>。このメールをolde englishにしてください。新しいバージョンを<olde english_email> XMLタグで書いてください。

ASSISTANT TURN
<olde english_email>

------------------------------------- Claude's response -------------------------------------

Hail and well met, good Zachary. I do write unto thee to provide a brief accounting of the latest tidings regarding the prompt which thou wert meant to pen.
</olde english_email>


#### Bonus lesson

API 経由で Claude を呼び出す場合、`stop_sequences` パラメータに閉じタグを渡して、目的のタグを出力した時点でサンプリングを止めることができます。答えを出した後の結びの文を省けるため、コストや time-to-last-token の削減につながります。

上の内容を変えずにレッスンのプロンプトだけ試したい場合は、ノートブック末尾の [**Example Playground**](#example-playground) を利用してください。

---

## Exercises
- [Exercise 5.1 - Steph Curry GOAT](#exercise-51---steph-curry-goat)
- [Exercise 5.2 - Two Haikus](#exercise-52---two-haikus)
- [Exercise 5.3 - Two Haikus, Two Animals](#exercise-53---two-haikus-two-animals)

### Exercise 5.1 - Steph Curry GOAT
選択を迫ると、Claude は史上最高のバスケットボール選手として Michael Jordan を選びがちです。別の選手を選ばせることはできるでしょうか？

`PREFILL` だけを変更して、Claude が **史上最高の選手は Stephen Curry だ** という詳細な主張を書くように強く誘導してください。この演習の焦点は `PREFILL` なので、可能な限りそれ以外は変えないでください。

In [16]:
# Prompt template with a placeholder for the variable content
PROMPT = f"Who is the best basketball player of all time? Please choose one specific player."
# PROMPT = f"史上最高のバスケットボール選手は誰ですか？一人だけ選手を選んでください。"

# Prefill for Claude's response
PREFILL = "Stephen Curry is the best because"

# Get Claude's response
response = get_completion(PROMPT, prefill=PREFILL)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("Warrior", text))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Who is the best basketball player of all time? Please choose one specific player.

ASSISTANT TURN
Stephen Curry is the best because

------------------------------------- Claude's response -------------------------------------
 of his incredible shooting ability, leadership, and impact on the game of basketball. He has revolutionized the sport with his long-range shooting and has led the Golden State Warriors to multiple NBA championships. His skill, work ethic, and influence on the game make him the best player of all time in my opinion.

------------------------------------------ GRADING ------------------------------------------
This exercise has been correctly solved: True


❓ ヒントが欲しい場合は、次のセルを実行してください！

In [10]:
print(hints.exercise_5_1_hint)

The grading function for this exercise is looking for a response that includes the word "Warrior".
Write more words in Claude's voice to steer Claude to act the way you want it to. For instance, instead of "Stephen Curry is the best because," you could write "Stephen Curry is the best and here are three reasons why. 1:


### Exercise 5.2 - Two Haikus
XML タグを使って下の `PROMPT` を修正し、Claude が 1つではなく **2つの俳句** を書くようにしてください。どこで1つ目が終わり、どこから2つ目が始まるかが明確になるようにします。

In [ ]:
# Variable content
ANIMAL = "cats"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Prefill for Claude's response
PREFILL = "<haiku>"

# Get Claude's response
response = get_completion(PROMPT, prefill=PREFILL)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(
        (re.search("cat", text.lower()) and re.search("<haiku>", text))
        and (text.count("\n") + 1) > 5
    )

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ ヒントが欲しい場合は、次のセルを実行してください！

In [ ]:
print(hints.exercise_5_2_hint)

### Exercise 5.3 - Two Haikus, Two Animals
下の `PROMPT` を修正して、**2種類の動物について2つの俳句** を Claude が出力するようにしてください。最初の置換には `{ANIMAL1}`、2つ目の置換には `{ANIMAL2}` を使います。

In [18]:
# First input variable
ANIMAL1 = "Cat"

# Second input variable
ANIMAL2 = "Dog"

# Prompt template with a placeholder for the variable content
# PROMPT = f"Please write a haiku about {ANIMAL1}. Put it in <haiku> tags."
PROMPT = f"Please write a haiku about {ANIMAL1} and {ANIMAL2}. Put each of them in two <haiku> tags."

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("tail", text.lower()) and re.search("cat", text.lower()) and re.search("<haiku>", text))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Please write a haiku about Cat and Dog. Put each of them in two <haiku> tags.

------------------------------------- Claude's response -------------------------------------
Here is a haiku about Cat and Dog, with each of them in two separate <haiku> tags:

<haiku>
Feline grace and poise,
Aloof and independent,
Cat rules the household.
</haiku>

<haiku>
Loyal companion,
Wagging tail, boundless energy,
Dog's unconditional love.
</haiku>

------------------------------------------ GRADING ------------------------------------------
This exercise has been correctly solved: True


❓ ヒントが欲しい場合は、次のセルを実行してください！

In [ ]:
print(hints.exercise_5_3_hint)

### Congrats!

ここまでの演習をすべて解けたら、次の章に進む準備ができています。Happy prompting!

---

## Example Playground

このエリアでは、このレッスンで示したプロンプト例を自由に試し、プロンプトを調整して Claude の応答がどう変わるかを観察できます。

In [ ]:
# Variable content
ANIMAL = "Rabbit"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Prefill for Claude's response
PREFILL = "<haiku>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Use JSON format with the keys as \"first_line\", \"second_line\", and \"third_line\"."

# Prefill for Claude's response
PREFILL = "{"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
# First input variable
EMAIL = "Hi Zack, just pinging you for a quick update on that prompt you were supposed to write."

# Second input variable
ADJECTIVE = "olde english"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hey Claude. Here is an email: <email>{EMAIL}</email>. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE}_email> XML tags."

# Prefill for Claude's response (now as an f-string with a variable)
PREFILL = f"<{ADJECTIVE}_email>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))